In [6]:
import duckdb
import pandas as pd
from datetime import datetime, timedelta
import os


db_file = 'local_baseball_games.duckdb'
conn = duckdb.connect(database=db_file)


csv_file_path = 'gameinfo.csv' 

print(f"Attempting to load data from {csv_file_path} into DuckDB table 'game_results'...")

try:
    conn.execute(f"""
        CREATE OR REPLACE TABLE game_results AS
        SELECT
            gid AS game_id,
            strptime(date::VARCHAR, '%Y%m%d')::DATE AS game_finish_date,
            visteam,
            hometeam,
            vruns AS vscore,
            hruns AS hscore,
            wteam AS winning_team,
            lteam AS losing_team,
            season
        FROM read_csv_auto('{csv_file_path}');
    """)
    print("Data successfully loaded from CSV into 'game_results' table.")

except duckdb.Error as e:
    print(f"Error loading CSV into DuckDB: {e}")
    print(f"Please ensure '{csv_file_path}' exists and is a valid CSV file.")
    print("Also check the 'date', 'gid', 'vruns', 'hruns', 'wteam', 'lteam', and 'season' column formats.")
    conn.close()
    exit()

class Game:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

    def __repr__(self):
        return (f"Game(ID: {getattr(self, 'game_id', 'N/A')}, "
                f"Date: {getattr(self, 'game_finish_date', 'N/A').strftime('%Y-%m-%d') if hasattr(self, 'game_finish_date') and self.game_finish_date else 'N/A'})")

def load_and_group_games_by_date(
    duckdb_connection: duckdb.DuckDBPyConnection,
    table_name: str = "game_results",
    date_column_name: str = "game_finish_date",
    start_year: int = 1901,
    end_year: int = datetime.now().year,
    cache_dir: str = "cached_game_data"
) -> dict[datetime.date, list[Game]]:

    os.makedirs(cache_dir, exist_ok=True)
    cache_file = os.path.join(cache_dir, f"games_{start_year}_to_{end_year}.parquet")

    full_df = pd.DataFrame() # Initialize empty DataFrame

    if os.path.exists(cache_file):
        print(f"Loading games from local cache: {cache_file}")
        try:
            full_df = pd.read_parquet(cache_file)
            full_df[date_column_name] = pd.to_datetime(full_df[date_column_name])
            print(f"Successfully loaded {len(full_df)} games from cache.")
        except Exception as e:
            print(f"Error loading Parquet cache, regenerating: {e}")
            os.remove(cache_file) 
    
    if full_df.empty: 
        print(f"Cache not found or invalid. Fetching all games from LOCAL DuckDB ({start_year}-{end_year})...")
        query = f"""
            SELECT *
            FROM {table_name}
            WHERE STRFTIME({date_column_name}::DATE, '%Y')::INTEGER BETWEEN {start_year} AND {end_year}
            ORDER BY {date_column_name}::DATE ASC, game_id ASC
        """
        try:
            full_df = duckdb_connection.execute(query).fetchdf()
            print(f"Successfully fetched {len(full_df)} games from DuckDB.")
            full_df[date_column_name] = pd.to_datetime(full_df[date_column_name])
            
            print(f"Saving fetched data to cache: {cache_file}")
            full_df.to_parquet(cache_file)
            print("Cache saved.")

        except duckdb.Error as e:
            print(f"Error fetching all games from DuckDB: {e}")
            return {}

    print("Grouping games by date...")
    games_by_date = {}
    for date_obj, group_df in full_df.groupby(full_df[date_column_name].dt.date):
        games_on_this_date = [Game(**row_dict) for row_dict in group_df.to_dict(orient='records')]
        games_by_date[date_obj] = games_on_this_date
    
    print(f"Finished grouping. Total unique dates: {len(games_by_date)}")
    return games_by_date

def daily_game_iterator_from_dict(games_by_date: dict[datetime.date, list[Game]]):
    sorted_dates = sorted(games_by_date.keys())
    for date in sorted_dates:
        yield date, games_by_date[date]

def weekly_game_iterator_from_dict(games_by_date: dict[datetime.date, list[Game]]):
    sorted_dates = sorted(games_by_date.keys())
    if not sorted_dates: return
    current_week_start = sorted_dates[0]
    current_week_games = []
    for game_date in sorted_dates:
        if game_date < current_week_start + timedelta(days=7):
            current_week_games.extend(games_by_date[game_date])
        else:
            yield current_week_start, current_week_games
            current_week_start = game_date
            while current_week_start < game_date: current_week_start += timedelta(days=1)
            current_week_games = games_by_date[game_date]
    if current_week_games:
        yield current_week_start, current_week_games


if __name__ == "__main__":
    current_year = datetime.now().year
    output_filename = "all_processed_games.parquet" 

    
    all_games_by_date = load_and_group_games_by_date(
        conn,
        start_year=1901,
        end_year=current_year
    )

    if not all_games_by_date:
        print("No games loaded. Exiting.")
        conn.close()
        exit()

    print("\n--- Accumulating all game objects for final output ---")
    all_game_objects_for_output = []

    
    daily_iter = daily_game_iterator_from_dict(all_games_by_date)
    for date, games_for_day in daily_iter:
        all_game_objects_for_output.extend(games_for_day)

    
    if all_game_objects_for_output:
        print(f"Collected {len(all_game_objects_for_output)} game objects.")
        output_df = pd.DataFrame([vars(game) for game in all_game_objects_for_output])
        
        if 'game_finish_date' in output_df.columns:
            output_df['game_finish_date'] = pd.to_datetime(output_df['game_finish_date'])

        print(f"Saving all processed game data to '{output_filename}'...")
        try:
            output_df.to_parquet(output_filename, index=False) 
            print(f"Successfully saved all data to '{output_filename}'.")
        except Exception as e:
            print(f"Error saving to Parquet: {e}")

        csv_output_filename = "all_processed_games.csv"
        print(f"Saving all processed game data to '{csv_output_filename}'...")
        try:
            output_df.to_csv(csv_output_filename, index=False)
            print(f"Successfully saved all data to '{csv_output_filename}'.")
        except Exception as e:
            print(f"Error saving to CSV: {e}")

    else:
        print("No game data collected for output.")


    conn.close()
    print("\nDuckDB connection closed.")

Attempting to load data from gameinfo.csv into DuckDB table 'game_results'...
Data successfully loaded from CSV into 'game_results' table.
Loading games from local cache: cached_game_data/games_1901_to_2025.parquet
Successfully loaded 219369 games from cache.
Grouping games by date...
Finished grouping. Total unique dates: 23042

--- Accumulating all game objects for final output ---
Collected 219369 game objects.
Saving all processed game data to 'all_processed_games.parquet'...
Successfully saved all data to 'all_processed_games.parquet'.
Saving all processed game data to 'all_processed_games.csv'...
Successfully saved all data to 'all_processed_games.csv'.

DuckDB connection closed.


Loading the entire processed dataset from 'all_processed_games.parquet'...
Successfully loaded 219369 games into a Pandas DataFrame.

--- First 5 rows (default preview) ---
        game_id game_finish_date visteam hometeam  vscore  hscore  \
0  PHI190104180       1901-04-18     BRO      PHI      12       7   
1  BRO190104190       1901-04-19     PHI      BRO       2      10   
2  BSN190104190       1901-04-19     NY1      BSN       0       7   
3  SLN190104190       1901-04-19     CHN      SLN       8       7   
4  CIN190104200       1901-04-20     PIT      CIN       4       2   

  winning_team losing_team  season  
0          BRO         PHI    1901  
1          BRO         PHI    1901  
2          BSN         NY1    1901  
3          CHN         SLN    1901  
4          PIT         CIN    1901  

--- Last 5 rows ---
             game_id game_finish_date visteam hometeam  vscore  hscore  \
219364  LAN202410250       2024-10-25     NYA      LAN       3       6   
219365  LAN202410260 